# on day level: find the regression coef with minimum SSE. (Means most correlated from past few days). Then drill down to the next day's 15 min data and  apply pairs trade on violation of returns ratio (coef)


In [2]:
import pandas as pd
import eikon as ek

ek.set_app_key('APP_KEY')
# ['SBI.NS','AXBK.NS','HDBK.NS','KTKM.NS','ICBK.NS']

alldata = ek.get_timeseries(rics=['SBI.NS','AXBK.NS','HDBK.NS','KTKM.NS','ICBK.NS'],fields=['CLOSE'],
                            start_date='2019-01-04',
                            end_date='2020-08-30',
                            interval='daily',corax='adjusted')


alldata.head()

/Users/vipultanwar/Documents/kite/kite/lib/python3.8/site-packages/eikon/time_series.py:282: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  datapoints = pd.np.array(timeseries['dataPoints'])
/Users/vipultanwar/Documents/kite/kite/lib/python3.8/site-packages/eikon/time_series.py:284: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  timestamps = pd.np.array([tz_replacer(value) for value in datapoints[:, timestamp_index]],
/Users/vipultanwar/Documents/kite/kite/lib/python3.8/site-packages/eikon/time_series.py:286: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  datapoints = pd.np.delete(datapoints, pd.np.s_[timestamp_index],
/Users/vipultanwar/Documents/kite/kite/lib/python3.8/site-packages/eikon/time_series.py:209: FutureWarning: T

CLOSE,SBI.NS,AXBK.NS,HDBK.NS,KTKM.NS,ICBK.NS
Date,,,,,
2019-01-04,297.65,619.60,1056.373572,1247.95,365.20
2019-01-07,296.30,637.45,1057.970018,1246.60,367.70
2019-01-08,305.30,650.90,1049.164618,1228.90,380.15
2019-01-09,305.30,670.10,1056.049294,1238.25,382.25
2019-01-10,305.55,663.25,1051.908511,1219.55,379.50


In [ ]:
(alldata[-30:]).corr()

In [ ]:
Axis_ICICI = ek.get_timeseries(rics=['AXBK.NS','ICBK.NS'],fields=['CLOSE'],
                            start_date='2018-01-04',
                            end_date='2020-08-30',
                            interval='daily',corax='adjusted')
Axis_ICICI.head()

In [ ]:



returns = returns[1:]

returns.head()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import train_test_split 

from sklearn import metrics


X = returns['AXBK.NS'].values.reshape(-1,1)
y = returns['ICBK.NS'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

epsilon = 1
huber = HuberRegressor(alpha=0.0, epsilon=epsilon)
huber.fit(X_train, y_train)


huber.su


In [ ]:
y_pred = huber.predict(X_test)
df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
df

In [ ]:
plt.scatter(df['Actual'],df['Predicted'])

In [3]:
import matplotlib.pyplot as plt
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import train_test_split 

from sklearn import metrics
import pandas as pd
import eikon as ek

ek.set_app_key('1b68381d8f684b0183be832bab589097fd9942f2')
# ['SBI.NS','ICBK.NS','HDBK.NS','KTKM.NS','SBIN.NS']




ICICI_SBI = ek.get_timeseries(rics=['ICBK.NS','SBIN.NS'],fields=['CLOSE'],
                            start_date='2019-09-04',
                            end_date='2020-09-03',
                            interval='daily',corax='adjusted')

returns = (ICICI_SBI  - ICICI_SBI.shift(1))/ICICI_SBI.shift(1)
returns = returns.fillna(0)
from sklearn.linear_model import LinearRegression

returns['Huber_Coef'] = 0.0
Samples_Taken = 10
returns['score'] = 0.0
returns['SSE'] = 0.0

for i in range(Samples_Taken,len(returns)):
    X = returns['ICBK.NS'][i-Samples_Taken:i].values.reshape(-1,1)
    y = returns['SBIN.NS'][i-Samples_Taken:i].values.reshape(-1,1)
    print(X,y)
    huber = HuberRegressor()
    huber.fit(X, y)
    returns['Huber_Coef'][i] = huber.coef_
    returns['score'][i] = huber.score(X, y)
    returns['SSE'][i] = metrics.mean_squared_error(X, y)
    
returns.head()

[[ 0.        ]
 [-0.02138365]
 [ 0.00604113]
 [ 0.01047656]
 [-0.00214945]
 [ 0.02052712]
 [ 0.02657065]
 [-0.00302371]
 [-0.02790246]
 [-0.00324473]] [[ 0.        ]
 [ 0.00694366]
 [ 0.00314807]
 [ 0.00575335]
 [ 0.00575759]
 [ 0.00036933]
 [ 0.00180905]
 [ 0.00401695]
 [-0.01673763]
 [ 0.00481559]]
[[-0.02138365]
 [ 0.00604113]
 [ 0.01047656]
 [-0.00214945]
 [ 0.02052712]
 [ 0.02657065]
 [-0.00302371]
 [-0.02790246]
 [-0.00324473]
 [-0.03192688]] [[ 0.00694366]
 [ 0.00314807]
 [ 0.00575335]
 [ 0.00575759]
 [ 0.00036933]
 [ 0.00180905]
 [ 0.00401695]
 [-0.01673763]
 [ 0.00481559]
 [-0.00572129]]
[[ 0.00604113]
 [ 0.01047656]
 [-0.00214945]
 [ 0.02052712]
 [ 0.02657065]
 [-0.00302371]
 [-0.02790246]
 [-0.00324473]
 [-0.03192688]
 [ 0.07992757]] [[ 0.00314807]
 [ 0.00575335]
 [ 0.00575759]
 [ 0.00036933]
 [ 0.00180905]
 [ 0.00401695]
 [-0.01673763]
 [ 0.00481559]
 [-0.00572129]
 [ 0.04648208]]
[[ 0.01047656]
 [-0.00214945]
 [ 0.02052712]
 [ 0.02657065]
 [-0.00302371]
 [-0.02790246]
 [-0

 [ 0.02655687]] [[-0.00304167]
 [ 0.00196617]
 [-0.00192848]
 [-0.00620339]
 [ 0.00098919]
 [ 0.00361208]
 [-0.00740187]
 [-0.00437846]
 [ 0.00395108]
 [-0.01204613]]
[[-0.00172768]
 [ 0.01784749]
 [-0.00456961]
 [ 0.02636917]
 [-0.00447264]
 [ 0.02277714]
 [ 0.01501686]
 [-0.02224235]
 [ 0.02655687]
 [ 0.0024065 ]] [[ 0.00196617]
 [-0.00192848]
 [-0.00620339]
 [ 0.00098919]
 [ 0.00361208]
 [-0.00740187]
 [-0.00437846]
 [ 0.00395108]
 [-0.01204613]
 [ 0.00225155]]
[[ 0.01784749]
 [-0.00456961]
 [ 0.02636917]
 [-0.00447264]
 [ 0.02277714]
 [ 0.01501686]
 [-0.02224235]
 [ 0.02655687]
 [ 0.0024065 ]
 [-0.0030009 ]] [[-0.00192848]
 [-0.00620339]
 [ 0.00098919]
 [ 0.00361208]
 [-0.00740187]
 [-0.00437846]
 [ 0.00395108]
 [-0.01204613]
 [ 0.00225155]
 [ 0.00190088]]
[[-0.00456961]
 [ 0.02636917]
 [-0.00447264]
 [ 0.02277714]
 [ 0.01501686]
 [-0.02224235]
 [ 0.02655687]
 [ 0.0024065 ]
 [-0.0030009 ]
 [-0.00973212]] [[-0.00620339]
 [ 0.00098919]
 [ 0.00361208]
 [-0.00740187]
 [-0.00437846]
 [ 

[[-0.0087896 ]
 [-0.00036948]
 [-0.004343  ]
 [ 0.01976798]
 [-0.00991991]
 [-0.00928394]
 [-0.00398961]
 [ 0.0071728 ]
 [-0.00323714]
 [-0.02440382]] [[ 0.00425109]
 [-0.00187758]
 [-0.00143649]
 [ 0.00434991]
 [ 0.00017051]
 [-0.00494408]
 [-0.0007196 ]
 [ 0.00802414]
 [-0.00166689]
 [-0.01908202]]
[[-0.00036948]
 [-0.004343  ]
 [ 0.01976798]
 [-0.00991991]
 [-0.00928394]
 [-0.00398961]
 [ 0.0071728 ]
 [-0.00323714]
 [-0.02440382]
 [-0.00532623]] [[-0.00187758]
 [-0.00143649]
 [ 0.00434991]
 [ 0.00017051]
 [-0.00494408]
 [-0.0007196 ]
 [ 0.00802414]
 [-0.00166689]
 [-0.01908202]
 [ 0.00507173]]
[[-0.004343  ]
 [ 0.01976798]
 [-0.00991991]
 [-0.00928394]
 [-0.00398961]
 [ 0.0071728 ]
 [-0.00323714]
 [-0.02440382]
 [-0.00532623]
 [ 0.00583286]] [[-0.00143649]
 [ 0.00434991]
 [ 0.00017051]
 [-0.00494408]
 [-0.0007196 ]
 [ 0.00802414]
 [-0.00166689]
 [-0.01908202]
 [ 0.00507173]
 [-0.0042512 ]]
[[ 0.01976798]
 [-0.00991991]
 [-0.00928394]
 [-0.00398961]
 [ 0.0071728 ]
 [-0.00323714]
 [-0

[[-0.00466027]
 [ 0.01086244]
 [ 0.01769338]
 [-0.01511014]
 [ 0.00887246]
 [-0.00769513]
 [-0.00073855]
 [ 0.00665188]
 [ 0.00403818]
 [-0.03135283]] [[-0.00394411]
 [ 0.00026846]
 [-0.00191224]
 [-0.00440321]
 [-0.00729237]
 [-0.01013468]
 [-0.00439772]
 [ 0.0135275 ]
 [ 0.00789922]
 [-0.01732991]]
[[ 0.01086244]
 [ 0.01769338]
 [-0.01511014]
 [ 0.00887246]
 [-0.00769513]
 [-0.00073855]
 [ 0.00665188]
 [ 0.00403818]
 [-0.03135283]
 [ 0.00207606]] [[ 0.00026846]
 [-0.00191224]
 [-0.00440321]
 [-0.00729237]
 [-0.01013468]
 [-0.00439772]
 [ 0.0135275 ]
 [ 0.00789922]
 [-0.01732991]
 [-0.00979752]]
[[ 0.01769338]
 [-0.01511014]
 [ 0.00887246]
 [-0.00769513]
 [-0.00073855]
 [ 0.00665188]
 [ 0.00403818]
 [-0.03135283]
 [ 0.00207606]
 [-0.01365477]] [[-0.00191224]
 [-0.00440321]
 [-0.00729237]
 [-0.01013468]
 [-0.00439772]
 [ 0.0135275 ]
 [ 0.00789922]
 [-0.01732991]
 [-0.00979752]
 [-0.01312318]]
[[-0.01511014]
 [ 0.00887246]
 [-0.00769513]
 [-0.00073855]
 [ 0.00665188]
 [ 0.00403818]
 [-0

[[-0.02192579]
 [ 0.07446308]
 [-0.03516195]
 [-0.00998034]
 [ 0.04475332]
 [ 0.09809942]
 [-0.03794435]
 [-0.08151121]
 [ 0.01235498]
 [ 0.05060277]] [[ 0.00055784]
 [ 0.03049277]
 [-0.0278009 ]
 [ 0.04516267]
 [-0.00450543]
 [ 0.01917301]
 [-0.0052077 ]
 [-0.0251603 ]
 [ 0.0068687 ]
 [ 0.01418117]]
[[ 0.07446308]
 [-0.03516195]
 [-0.00998034]
 [ 0.04475332]
 [ 0.09809942]
 [-0.03794435]
 [-0.08151121]
 [ 0.01235498]
 [ 0.05060277]
 [-0.05128205]] [[ 0.03049277]
 [-0.0278009 ]
 [ 0.04516267]
 [-0.00450543]
 [ 0.01917301]
 [-0.0052077 ]
 [-0.0251603 ]
 [ 0.0068687 ]
 [ 0.01418117]
 [-0.00998777]]
[[-0.03516195]
 [-0.00998034]
 [ 0.04475332]
 [ 0.09809942]
 [-0.03794435]
 [-0.08151121]
 [ 0.01235498]
 [ 0.05060277]
 [-0.05128205]
 [ 0.03897267]] [[-0.0278009 ]
 [ 0.04516267]
 [-0.00450543]
 [ 0.01917301]
 [-0.0052077 ]
 [-0.0251603 ]
 [ 0.0068687 ]
 [ 0.01418117]
 [-0.00998777]
 [ 0.0042413 ]]
[[-0.00998034]
 [ 0.04475332]
 [ 0.09809942]
 [-0.03794435]
 [-0.08151121]
 [ 0.01235498]
 [ 0

[[ 0.02509017]
 [ 0.01560349]
 [ 0.02199126]
 [ 0.02697126]
 [ 0.02425373]
 [-0.02522068]
 [ 0.0268794 ]
 [ 0.00727884]
 [-0.03126737]
 [ 0.01276718]] [[ 0.00956119]
 [ 0.02175352]
 [ 0.02574273]
 [ 0.00284113]
 [ 0.00692532]
 [ 0.00429855]
 [ 0.0140856 ]
 [-0.0071752 ]
 [ 0.00830918]
 [ 0.00256803]]
[[ 0.01560349]
 [ 0.02199126]
 [ 0.02697126]
 [ 0.02425373]
 [-0.02522068]
 [ 0.0268794 ]
 [ 0.00727884]
 [-0.03126737]
 [ 0.01276718]
 [-0.03328612]] [[ 0.02175352]
 [ 0.02574273]
 [ 0.00284113]
 [ 0.00692532]
 [ 0.00429855]
 [ 0.0140856 ]
 [-0.0071752 ]
 [ 0.00830918]
 [ 0.00256803]
 [-0.00718737]]
[[ 0.02199126]
 [ 0.02697126]
 [ 0.02425373]
 [-0.02522068]
 [ 0.0268794 ]
 [ 0.00727884]
 [-0.03126737]
 [ 0.01276718]
 [-0.03328612]
 [ 0.00864469]] [[ 0.02574273]
 [ 0.00284113]
 [ 0.00692532]
 [ 0.00429855]
 [ 0.0140856 ]
 [-0.0071752 ]
 [ 0.00830918]
 [ 0.00256803]
 [-0.00718737]
 [-0.00804806]]
[[ 0.02697126]
 [ 0.02425373]
 [-0.02522068]
 [ 0.0268794 ]
 [ 0.00727884]
 [-0.03126737]
 [ 0

[[-0.02276584]
 [ 0.00072348]
 [-0.00332562]
 [ 0.02654867]
 [ 0.02600339]
 [ 0.04352617]
 [ 0.00607181]
 [ 0.02925741]
 [-0.02664117]
 [-0.06102672]] [[-0.00411294]
 [-0.0085203 ]
 [ 0.00345242]
 [ 0.01398654]
 [ 0.00851958]
 [ 0.00285244]
 [-0.00495934]
 [ 0.00809909]
 [-0.00621637]
 [-0.01499799]]
[[ 0.00072348]
 [-0.00332562]
 [ 0.02654867]
 [ 0.02600339]
 [ 0.04352617]
 [ 0.00607181]
 [ 0.02925741]
 [-0.02664117]
 [-0.06102672]
 [-0.01785216]] [[-0.0085203 ]
 [ 0.00345242]
 [ 0.01398654]
 [ 0.00851958]
 [ 0.00285244]
 [-0.00495934]
 [ 0.00809909]
 [-0.00621637]
 [-0.01499799]
 [ 0.00014855]]
[[-0.00332562]
 [ 0.02654867]
 [ 0.02600339]
 [ 0.04352617]
 [ 0.00607181]
 [ 0.02925741]
 [-0.02664117]
 [-0.06102672]
 [-0.01785216]
 [-0.00298211]] [[ 0.00345242]
 [ 0.01398654]
 [ 0.00851958]
 [ 0.00285244]
 [-0.00495934]
 [ 0.00809909]
 [-0.00621637]
 [-0.01499799]
 [ 0.00014855]
 [ 0.0025621 ]]
[[ 0.02654867]
 [ 0.02600339]
 [ 0.04352617]
 [ 0.00607181]
 [ 0.02925741]
 [-0.02664117]
 [-0

CLOSE,ICBK.NS,SBIN.NS,Huber_Coef,score,SSE
Date,,,,,
2019-09-04,0.000000,0.000000,0.0,0.0,0.0
2019-09-05,-0.021384,0.006944,0.0,0.0,0.0
2019-09-06,0.006041,0.003148,0.0,0.0,0.0
2019-09-09,0.010477,0.005753,0.0,0.0,0.0
2019-09-11,-0.002149,0.005758,0.0,0.0,0.0


In [4]:
returns.tail()

CLOSE,ICBK.NS,SBIN.NS,Huber_Coef,score,SSE
Date,,,,,
2020-08-28,0.044620,0.002160,0.085294,0.106555,0.000223
2020-08-31,-0.036856,-0.034582,-0.008126,-0.003597,0.000397
2020-09-01,-0.009250,0.014076,0.166561,0.316708,0.000390
2020-09-02,0.003709,0.008520,0.061385,0.079523,0.000402
2020-09-03,-0.024592,0.010173,0.033725,0.032719,0.000402


In [2]:

import warnings
warnings.filterwarnings('ignore')

def DayTrade(sellsym, buysym,date,period,slope):
    
    stt_date = date + ' 3:46:00'
    end_date = date + ' 10:30:00'

    data = ek.get_timeseries(rics=[sellsym,buysym],fields=['CLOSE'],
                                start_date=stt_date,
                                end_date=end_date,
                                interval='minute',corax='adjusted')

    data_15 = pd.DataFrame(columns=data.columns)
    data_15[sellsym] = data.resample(period).ohlc()[sellsym,'close']
    data_15[buysym] = data.resample(period).ohlc()[buysym,'close']

    datarets = data_15 - data_15.shift(1)
    datarets.fillna(0)


    buy_timestamps = []
    sell_timestamps = []
    for i,row in datarets.iterrows():
        if(row[sellsym] > 0):
            if(row[buysym]/row[sellsym] < (slope-1) ):
                buy_timestamps.append(i)

    for i,row in datarets.iterrows():
        if(row[sellsym] > 0):
            if(row[buysym]/row[sellsym] > (slope+1) ):
                sell_timestamps.append(i)

    buys = pd.DataFrame(index=buy_timestamps)
    buys['Action'] = 1
    sells = pd.DataFrame(index=sell_timestamps)
    sells['Action'] = -1

    Actions = (buys.append(sells))
#     print(sells)
    
    
    
    data_15 = pd.concat([data_15.fillna(method='ffill'),Actions],axis=1,join='outer')
#     data_15 = pd.concat([data_15.fillna(method='ffill'),sells],axis=1,join='outer')

    data_15.Action[0] = 0
#     data_15.sell[0] = 0

#     data_15.buy = data_15.buy.fillna(method='pad')
#     data_15.sell = data_15.sell.fillna(method='pad')
    data_15['Spread'] =   data_15[sellsym] - data_15[buysym]
    data_15['nUnits'] =   data_15.Action.fillna(method='pad')
    data_15['SpreadRets'] = (data_15.Spread - data_15.Spread.shift(1))/data_15.Spread.shift(1)
    data_15['port rets'] = data_15['SpreadRets'] * data_15['nUnits'].shift(0)
    data_15['cum rets'] = data_15['port rets'].cumsum() 
#     print(data_15.head(20))

#     plt.plot(data_15['Spread'])   
#     plt.show()

#     plt.plot(data_15['cum rets'])
#     plt.show()
    
#     data_15.to_csv('daytrade.csv')
    return data_15['cum rets'].iloc[-1], data_15['cum rets'].min(),len(Actions)


In [173]:
returns_by_sse = returns_by_sse.reset_index(drop=True)

returns_by_sse.to_csv('returns_by_sse.csv')

In [174]:
import time
# returns_by_sse = returns_by_sse.reset_index(drop=True)
returns = returns_by_sse
returns['daytraderets'] = 0.000 
returns['maxdrawdown'] = 0.000 
returns['TotalTrades'] = 0

for i,row in returns.iterrows():
    try:
        returns.loc[i,'daytraderets'],returns.loc[i,'maxdrawdown'],returns.loc[i,'TotalTrades'] = DayTrade(returns.loc[i,'S1'],returns.loc[i,'S2'],str(returns.loc[i,'Date']),'10min',row['Coef'])
    except:
        row['daytraderets'],row['maxdrawdown'],row['TotalTrades'] = 0.00 , 0.00 ,0 
    
    returns.to_csv('backtest_by_sse.csv')
    print(i)
    time.sleep(3)
returns.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224


2020-09-03 19:39:07,972 P[9035] [MainThread 4556389824] Error with HDBK.NS: No data available for the requested date range
2020-09-03 19:39:07,974 P[9035] [MainThread 4556389824] Error with KTKM.NS: No data available for the requested date range
2020-09-03 19:39:07,975 P[9035] [MainThread 4556389824] HDBK.NS: No data available for the requested date range | KTKM.NS: No data available for the requested date range | 


225
226
227
228
229
230
231
232


,Date,S1,S2,Coef,R_Sq,SSE,daytraderets,maxdrawdown,TotalTrades
0,2020-07-21,ICBK.NS,IDFB.NS,1.08665,0.770586,0.000150227,0.019491,-0.001440,15
1,2020-07-22,ICBK.NS,IDFB.NS,1.08597,0.797066,0.000150731,0.002052,-0.007688,9
2,2020-07-23,HDBK.NS,IDFB.NS,1.12064,0.791213,0.000172849,0.009090,0.000000,18
3,2020-07-24,HDBK.NS,IDFB.NS,1.07723,0.776146,0.000148562,0.006068,-0.007939,15
4,2020-07-27,HDBK.NS,IDFB.NS,1.10105,0.763697,0.000148169,-0.001366,-0.003033,13


In [ ]:
import os
import eikon as ek
import numpy as np
import pandas as pd
from matplotlib import pyplot
import time
import warnings
warnings.filterwarnings("ignore")


import datetime

# end_date = datetime.date.today().strftime("%Y-%m-%d")

# stt_date = (datetime.date.today() - datetime.timedelta(days=99)).strftime("%Y-%m-%d")


#############################

symdict = {}
symdict["symbols1"]  = ['AXBK.NS','BANH.NS','BOB.NS','FED.NS','HDBK.NS']
symdict["symbols2"]  = ['ICBK.NS','IDFB.NS','INBK.NS','KTKM.NS','PNBK.NS']
symdict["symbols3"] = ['RATB.NS','SBIN.NS']
#############################
ek.set_app_key('1b68381d8f684b0183be832bab589097fd9942f2')


# df = pd.DataFrame()

dfdict = {}
dfdict[0] = pd.DataFrame()
dfdict[1] = pd.DataFrame()
dfdict[2] = pd.DataFrame()
dfdict[3] = pd.DataFrame()
dfdict[4] = pd.DataFrame()
dfdict[5] = pd.DataFrame()



# df_open = pd.DataFrame()
for k in range(0,360,60):
    for i in range(1,4):
        key = "symbols" + str(i)
        stt_date = (datetime.date.today() - datetime.timedelta(days=60 + k + 1 )).strftime("%Y-%m-%d")
        end_date = (datetime.date.today() - datetime.timedelta(days= k + 2 )).strftime("%Y-%m-%d")
        print(stt_date,end_date,k/60,key)

        dfdict[k/60] = pd.concat([dfdict[k/60],ek.get_timeseries(rics=symdict[key],fields=['CLOSE'],start_date=stt_date,end_date=end_date,interval='daily',corax='adjusted')],axis = 1)
        time.sleep(3)

df = pd.DataFrame(columns=dfdict[0].columns)
for i in range(0,6):
    df = df.append(dfdict[i])
print('done_downloading')



In [172]:
returns = (df - df.shift(1))/df.shift(1)
returns = returns.fillna(0)

returns_by_sse = pd.DataFrame(columns=['Date','S1','S2','Coef','R_Sq','SSE'])
# returns['Coef'] = 0.0
Samples_Taken = 10
# returns['R_Sq'] = 0.0
# returns['SSE'] = 0.0
combinations = pd.DataFrame(columns=['Date','S1','S2','Coef','R_Sq','SSE'])
rownum = 0
for i in range(Samples_Taken,len(returns)-1):
    for k in range(len(returns.columns)):
        for j in range(k+1,len(returns.columns)):
            rownum = k*11 + j
            combinations.loc[rownum,'S1'],combinations.loc[rownum,'S2'] =  returns.columns[k],returns.columns[j]
            X = returns[returns.columns[k]][i-Samples_Taken:i].values.reshape(-1,1)
            y = returns[returns.columns[j]][i-Samples_Taken:i].values.reshape(-1,1)
            Reg = LinearRegression()
            Reg.fit(X, y)
            combinations.loc[rownum,'Coef'] = Reg.coef_[0][0]
            combinations.loc[rownum,'R_Sq'] = Reg.score(X, y)
            combinations.loc[rownum,'SSE'] =  metrics.mean_squared_error(X, y)
            combinations.loc[rownum,'Date'] = returns.index[i+1].date()
    returns_by_sse = returns_by_sse.append(combinations.sort_values(by='SSE',ascending=True).head(1))

    
#         print(returns.columns[k],returns.columns[j],Reg.coef_,Reg.score(X, y),metrics.mean_squared_error(X, y))
    
    
returns_by_sse


,Date,S1,S2,Coef,R_Sq,SSE
61,2020-07-21,ICBK.NS,IDFB.NS,1.08665,0.770586,0.000150227
61,2020-07-22,ICBK.NS,IDFB.NS,1.08597,0.797066,0.000150731
50,2020-07-23,HDBK.NS,IDFB.NS,1.12064,0.791213,0.000172849
50,2020-07-24,HDBK.NS,IDFB.NS,1.07723,0.776146,0.000148562
50,2020-07-27,HDBK.NS,IDFB.NS,1.10105,0.763697,0.000148169
...,...,...,...,...,...,...
55,2019-10-31,HDBK.NS,SBIN.NS,0.512778,0.250964,4.1204e-05
52,2019-11-01,HDBK.NS,KTKM.NS,1.02925,0.437002,4.99961e-05
55,2019-11-04,HDBK.NS,SBIN.NS,0.691567,0.380979,5.13022e-05
55,2019-11-05,HDBK.NS,SBIN.NS,0.627284,0.343867,4.87981e-05


In [157]:
returns_by_rsq.to_csv('by_rsq.csv')

In [145]:
combinations.sort_values(['R_Sq','Date'],ascending=[True,False])

,Date,S1,S2,Coef,R_Sq,SSE
10989,2019-11-26,FED.NS,INBK.NS,-0.000194129,1.45511e-08,0.000806753
1321,2020-08-18,AXBK.NS,BOB.NS,-0.000209208,3.34231e-08,0.00104247
1100,2020-08-12,HDBK.NS,SBIN.NS,6.85588e-05,3.86608e-08,0.000483477
12248,2019-12-23,HDBK.NS,ICBK.NS,-0.000115488,4.21799e-08,6.5948e-05
8261,2020-01-27,BANH.NS,BOB.NS,8.77105e-05,5.84057e-08,0.000696102
...,...,...,...,...,...,...
7973,2020-01-20,IDFB.NS,PNBK.NS,0.927721,0.999474,0.000724693
8044,2020-01-21,INBK.NS,RATB.NS,0.717004,0.999546,0.0480662
7947,2020-01-20,BOB.NS,PNBK.NS,0.95066,0.99966,0.000312262
8013,2020-01-21,BOB.NS,PNBK.NS,0.950133,0.999665,0.000312252
